In [2]:
pip install "numpy>=1.16.5,<1.23.0"

Note: you may need to restart the kernel to use updated packages.


In [2]:
# grader-required-cell
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import img_to_array, load_img

In [3]:
# atur lokasi
dir_dataset = "/kaggle/input/corn-or-maize-leaf-disease-dataset/data" # Mendefinisikan path dataset

dir_Blight = os.path.join("/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Blight")
dir_Common_rust = os.path.join("/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Common_Rust")
dir_Grey_leaf_spot = os.path.join("/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Gray_Leaf_Spot")
dir_Healthy = os.path.join("/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Healthy")

# # cek data per dir
total_Blight = len(os.listdir(dir_Blight))
total_Common_rust = len(os.listdir(dir_Common_rust))
total_Grey_leaf_spot = len(os.listdir(dir_Grey_leaf_spot))
total_Healthy  = len(os.listdir(dir_Healthy))

# cek total data
number_label = {}
total_data = 0
for i in os.listdir(dir_dataset):
    counting = len(os.listdir(os.path.join(dir_dataset, i)))
    number_label[i] = counting
    total_data += counting

print(f'''
Total dari Blight                    : {total_Blight}  data
Total dari Common Rust               : {total_Common_rust}  data
Total dari Grey Leaf Spot            : {total_Grey_leaf_spot}  data
Total dari Healty                    : {total_Healthy}  data
Total Dataset                        : {str(total_data)} data
''')


Total dari Blight                    : 1146  data
Total dari Common Rust               : 1306  data
Total dari Grey Leaf Spot            : 574  data
Total dari Healty                    : 1162  data
Total Dataset                        : 4188 data



In [4]:
#Train dan Validation dataset

val_size = 0.3 #menentukan ukuran validasi 40%

Train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = val_size
)

Validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = val_size
)

In [5]:
# Mengatur lebar dan tinggi gambar
img_width = 224
img_height = 224
channels = 3
batch_size = 16

# Train dan Validation generator  dengan mode categorical
Train_generator = Train_datagen.flow_from_directory(
    dir_dataset,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True,
    subset = "training"
)

Validation_generator = Validation_datagen.flow_from_directory(
    dir_dataset,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False,
    subset = "validation"
)

Found 2934 images belonging to 4 classes.
Found 1254 images belonging to 4 classes.


In [6]:
##### Pembuatan model
from keras import regularizers
# inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))

# for layer in inception_model.layers:
#   layer.trainable = False

model = tf.keras.models.Sequential(
    [    #  inception_model,
     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(img_width, img_height,3)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Flatten(),
     # Droupout
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.01)),
     tf.keras.layers.Dense(4, activation='softmax')
    ]
)

In [7]:
# Penggunaan Optimizer 'Adam' dengan learning rate 0.00146
Optimizer = tf.keras.optimizers.Adam(1e-5)
model.compile(optimizer=Optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 

In [8]:
# create callbacks
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs['accuracy'] >= 0.9 and logs['val_accuracy'] >= 0.8:
            print("\nReached 80% accuracy and testing accuracy so cancelling training!")
            self.model.stop_training = True

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 373248)            0         
                                                                 
 dropout (Dropout)           (None, 373248)            0

In [14]:
callbacks = myCallback()
history = model.fit(Train_generator,
                    validation_data = Validation_generator,
                    epochs = 60,
                    verbose = 1,
                    callbacks=callbacks)

Epoch 1/60
184/184 [==============================] - 20s 100ms/step - loss: 2.6956 - accuracy: 0.7277 - val_loss: 2.0507 - val_accuracy: 0.7839
Epoch 2/60
184/184 [==============================] - 17s 91ms/step - loss: 1.7005 - accuracy: 0.8078 - val_loss: 1.5774 - val_accuracy: 0.7679
Epoch 3/60
184/184 [==============================] - 17s 90ms/step - loss: 1.3580 - accuracy: 0.8258 - val_loss: 1.3294 - val_accuracy: 0.8054
Epoch 4/60
184/184 [==============================] - 17s 90ms/step - loss: 1.1781 - accuracy: 0.8473 - val_loss: 1.2066 - val_accuracy: 0.8102
Epoch 5/60
184/184 [==============================] - 17s 91ms/step - loss: 1.0605 - accuracy: 0.8599 - val_loss: 1.1467 - val_accuracy: 0.7903
Epoch 6/60
184/184 [==============================] - 17s 92ms/step - loss: 0.9712 - accuracy: 0.8654 - val_loss: 1.0395 - val_accuracy: 0.8198
Epoch 7/60
184/184 [==============================] - 16s 89ms/step - loss: 0.9097 - accuracy: 0.8657 - val_loss: 1.0021 - val_accuracy

In [12]:
Train_generator.class_indices

{'Blight': 0, 'Common_Rust': 1, 'Gray_Leaf_Spot': 2, 'Healthy': 3}

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np
# mengambil data acak 15
img_dir = "/kaggle/input/test-data-corn/test/gray leaf"
img_list = os.listdir(img_dir)
random_size = 6
img_sources = random.sample(img_list, random_size)

i = 1
total_high = []
for img_source in img_sources:
    img_path = os.path.join(img_dir, img_source)
    image_load = image.load_img(img_path, target_size=(img_width, img_height))
#     imgplot = plt.imshow(image_load)
    x = image.img_to_array(image_load)
    x = np.expand_dims(x, axis=0)
    probs = model.predict(x)


# img_source = "/kaggle/input/rice-leafs-disease-dataset/RiceLeafsDisease/validation/healthy/healthy_val (12).jpg"
# image_load = image.load_img(img_source, target_size = (img_width,img_height))
# imgplot = plt.imshow(image_load)
# x = image.img_to_array(image_load)
# x = np.expand_dims(x, axis=0)
# probs = cnn_model.predict(x)
    print (f'''iterasi ke - {i} : ''')
    # convert percent 
    probs = np.clip(probs,0,1)
    percent = probs * 100

    # print softmax probabilities
    high = np.argmax(probs)
    
    
    if high == 0:
        print ("Blight")
        print(f'''Akurasi : {np.round(percent[0,0],2)}''')
        total_high.append(high)
    elif high == 1:
        print("Common_Rust")
        print(f'''Akurasi : {np.round(percent[0,1],2)}''')
        total_high.append(high)
    elif high == 2:
        print("Gray_Leaf_Spot")
        print(f'''Akurasi : {np.round(percent[0,2],2)}''')
        total_high.append(high)
    else:
        print("Healthy")
        print(f'''Akurasi : {np.round(percent[0,3],2)}''')
        total_high.append(high)
    print("\n--------------------------------------------")
        
    i += 1


1/1 [==============================] - 0s 23ms/step
iterasi ke - 1 : 
Common_Rust
Akurasi : 100.0

--------------------------------------------
1/1 [==============================] - 0s 21ms/step
iterasi ke - 2 : 
Common_Rust
Akurasi : 100.0

--------------------------------------------
1/1 [==============================] - 0s 32ms/step
iterasi ke - 3 : 
Common_Rust
Akurasi : 100.0

--------------------------------------------
1/1 [==============================] - 0s 38ms/step
iterasi ke - 4 : 
Common_Rust
Akurasi : 100.0

--------------------------------------------
1/1 [==============================] - 0s 35ms/step
iterasi ke - 5 : 
Gray_Leaf_Spot
Akurasi : 100.0

--------------------------------------------
1/1 [==============================] - 0s 41ms/step
iterasi ke - 6 : 
Common_Rust
Akurasi : 100.0

--------------------------------------------


In [21]:
# initial
dir_name = os.path.basename(img_dir)
dir_name_trainVal = os.path.basename(os.path.dirname(img_dir))
# hitung list
count_data0 = total_high.count(0)
count_data1 = total_high.count(1)
count_data2 = total_high.count(2)
count_data3 = total_high.count(3)
count_data4 = total_high.count(4)
count_data5 = total_high.count(5)

list_count = [count_data0,count_data1,count_data2,count_data3,count_data4,count_data5]
max_list_count = max(list_count)

# kondisi
if dir_name == 'Blight':
    idx_name = count_data0
elif dir_name == 'Common_Rust':
    idx_name = count_data1
elif dir_name == 'Gray_Leaf_Spot':
    idx_name = count_data2
else:
    idx_name = count_data3
    

print(f'''\n*********************************************************************
Blight              : {count_data0} data
Common_Rust         : {count_data1} data
Gray_Leaf_Spot      : {count_data2} data
Healty              : {count_data2} data

Total data test     : {idx_name}/{random_size} data {dir_name} {dir_name_trainVal}

''')


*********************************************************************
Blight              : 0 data
Common_Rust         : 5 data
Gray_Leaf_Spot      : 1 data
Healty              : 1 data

Total data test     : 0/6 data gray leaf test




In [27]:
from tensorflow.keras.preprocessing import image

import numpy as np
img_source = "https://cals.cornell.edu/sites/default/files/styles/three_card_callout/public/2021-02/nclb2-10.11-250x376.gif?h=2aff6ec4&itok=VfogYnLk"
image_load = image.load_img(img_source, target_size = (img_width,img_height))
imgplot = plt.imshow(image_load)
x = image.img_to_array(image_load)
x = np.expand_dims(x, axis=0)
probs = model.predict(x)

highest = np.argmax(probs)

if highest == 0:
  print("Blight")
elif highest == 1:
  print("Common rust")
elif highest == 2:
  print("Gray Leaf Spot")
else:
  print("Healty")
  

# print softmax probabilities
print('Softmax Probabilities:')
print(probs)

FileNotFoundError: [Errno 2] No such file or directory: 'https://cals.cornell.edu/sites/default/files/styles/three_card_callout/public/2021-02/nclb2-10.11-250x376.gif?h=2aff6ec4&itok=VfogYnLk'

In [28]:
# Precission, F1 score, recall
import numpy as np
from sklearn.metrics import recall_score, f1_score, precision_score


val_preds = model.predict_generator(Validation_generator)
val_preds = np.argmax(val_preds, axis=-1)
val_true = Validation_generator.classes

# ==================== Precission
precision = precision_score(val_true, val_preds, average='weighted')
precision = round(float(precision), 2)

# ==================== Recall
recall = tf.keras.metrics.Recall()(val_true, val_preds)
recall = tf.reduce_mean(recall)
recall = round(float(recall), 2)

# ==================== F1 socre
f1 = f1_score(val_true, val_preds, average='weighted')
f1 = round(f1,2)

print(f'''
Hasil dari Precission, recall, dan F1 score sebagai berikut
--------------------------
Recall     : {recall} / {recall*100}%
--------------------------
Precission : {precision} / {precision*100}%
--------------------------
F1 score   : {f1} / {f1*100}%
--------------------------''')
# precision.dtype

/tmp/ipykernel_32/2251734997.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  val_preds = model.predict_generator(Validation_generator)



Hasil dari Precission, recall, dan F1 score sebagai berikut
--------------------------
Recall     : 0.92 / 92.0%
--------------------------
Precission : 0.81 / 81.0%
--------------------------
F1 score   : 0.8 / 80.0%
--------------------------


In [15]:
model.save('cnn_model_jagung_inp.h5')

In [16]:
import requests

# URL unduhan file H5
url = 'https://www.kaggle.com/code/muhamadnizam7/jagung/edit/run/130913261'

# Lokasi penyimpanan lokal
local_path = '/kaggle/working/cnn_model_jagung.h5'

# Buat objek permintaan HTTP
response = requests.get(url, stream=True)

# Buat variabel untuk menyimpan status pembatalan
cancel_flag = True

try:
    # Menghitung ukuran file
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # Ukuran blok unduhan

    # Menyimpan file secara bertahap
    with open(local_path, 'wb') as file:
        for data in response.iter_content(block_size):
            # Cek status pembatalan
            if cancel_flag:
                print("Unduhan dibatalkan.")
                break

            # Menulis data ke file
            file.write(data)

except KeyboardInterrupt:
    # Tangkap KeyboardInterrupt (misalnya jika Anda menekan tombol Ctrl+C)
    print("Unduhan dibatalkan melalui KeyboardInterrupt.")

finally:
    # Tutup koneksi
    response.close()

# Menghapus file yang sebagian terunduh jika pembatalan dilakukan
if cancel_flag:
    os.remove(local_path)

print("Unduhan selesai!")


Unduhan dibatalkan.
Unduhan selesai!
